In [1]:
import pandas as pd
import numpy as np

This work was inspired by [Ciara Ahern's](https://www.researchgate.net/profile/Ciara_Ahern) work on BER Default values.

# Load BERPublicsearch

- Downloaded on October 2020 from SEAI at https://ndber.seai.ie/BERResearchTool/Register/Register.aspx 
- Columns `Year_of_Construction`, `UValueWall` and `FirstWallType_Description` were extracted
- Data was saved as a `csv`, compressed into a `zip` file and uploaded to this repository at: https://github.com/codema-dev/berpublicsearch-defaults/tree/main/data 

In [2]:
ber = pd.read_csv("../data/BERPublicsearch.zip")

In [3]:
ber

,Year_of_Construction,UValueWall,FirstWallType_Description
0,1995,0.33,300mm Cavity
1,2002,0.55,300mm Filled Cavity
2,1978,0.94,300mm Cavity
3,1900,2.10,Stone
4,1983,0.54,Solid Mass Concrete
...,...,...,...
930331,1970,1.99,NaN
930332,1972,0.90,300mm Filled Cavity
930333,1950,2.02,Stone
930334,1998,0.69,NaN


# Bin individual buildings into the SEAI-specified age bands

- `age_bands` and `wall_uvalues` tables from [DEAP Manual 4.2.2 Appendix S](https://www.seai.ie/home-energy/building-energy-rating-ber/support-for-ber-assessors/domestic-ber-resources/) were copied into `csv` files and uploaded to this repository at: https://github.com/codema-dev/berpublicsearch-defaults/tree/main/data 


In [4]:
age_bands = pd.read_csv("../data/age-bands.csv")

In [5]:
age_bands

,Age band,Years of construction
0,A,before 1900
1,B,1900-1929
2,C,1930-1949
3,D,1950-1966
4,E,1967-1977
5,F,1978-1982
6,G,1983-1993
7,H,1994-1999
8,I,2000-2004
9,J,2005 -2009


In [6]:
ber["age_band_bin"] = pd.cut(
    ber["Year_of_Construction"],
    bins=[-np.inf, 1900, 1929, 1949, 1966, 1977, 1982, 1993, 1999, 2004, 2009, np.inf],
)

In [7]:
age_band_bins = [age_band_bin for age_band_bin in ber["age_band_bin"].unique().sort_values()]

In [8]:
age_bands = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K"]

In [9]:
age_band_map = {age_band_bin: age_band  for age_band, age_band_bin in zip(age_bands, age_band_bins)}

In [10]:
ber["age_band"] = ber["age_band_bin"].map(age_band_map)

In [11]:
ber["age_band"]

0         H
1         I
2         F
3         A
4         G
         ..
930331    E
930332    E
930333    D
930334    H
930335    K
Name: age_band, Length: 930336, dtype: category
Categories (11, object): ['A' < 'B' < 'C' < 'D' ... 'H' < 'I' < 'J' < 'K']

# Link each building to it's default wall U-value

In [12]:
wall_defaults_raw = pd.read_csv("../data/wall-uvalues.csv")

In [13]:
wall_defaults_raw

,Wall type,A,B,C,D,E,F,G,H,I,J,K
0,Stone,2.10,2.10,2.10,2.10,2.10,1.1,0.6,0.55,0.55,0.37,0.27
1,225mm solid brick,2.10,2.10,2.10,2.10,2.10,1.1,0.6,0.55,0.55,0.37,0.27
2,325mm solid brick,1.64,1.64,1.64,1.64,1.64,1.1,0.6,0.55,0.55,0.37,0.27
3,300mm cavity,2.10,1.78,1.78,1.78,1.78,1.1,0.6,0.55,0.55,0.37,0.27
4,300mm filled cavity,0.60,0.60,0.60,0.60,0.60,0.6,0.6,0.55,0.55,0.37,0.27
5,solid mass concrete,2.20,2.20,2.20,2.20,2.20,1.1,0.6,0.55,0.55,0.37,0.27
6,concrete hollow block,2.40,2.40,2.40,2.40,2.40,1.1,0.6,0.55,0.55,0.37,0.27
7,timber frame,2.50,1.90,1.90,1.10,1.10,1.1,0.6,0.55,0.55,0.37,0.27
8,Unknown,2.10,2.10,2.10,2.10,2.10,1.1,0.6,0.55,0.55,0.37,0.27
9,425 mm Cavity Wall,1.73,1.51,1.51,1.51,1.51,1.1,0.6,0.55,0.55,0.37,0.27


In [14]:
wall_defaults_pivoted = wall_defaults_raw.copy().melt(
    id_vars="Wall type",
    var_name="age_band",
    value_vars=age_bands,
    value_name="UValueWall_default"
)
wall_defaults_pivoted["wall_type"] = wall_defaults_pivoted["Wall type"].str.lower()
wall_defaults = wall_defaults_pivoted

In [15]:
wall_defaults

,Wall type,age_band,UValueWall_default,wall_type
0,Stone,A,2.10,stone
1,225mm solid brick,A,2.10,225mm solid brick
2,325mm solid brick,A,1.64,325mm solid brick
3,300mm cavity,A,2.10,300mm cavity
4,300mm filled cavity,A,0.60,300mm filled cavity
...,...,...,...,...
116,concrete hollow block,K,0.27,concrete hollow block
117,timber frame,K,0.27,timber frame
118,Unknown,K,0.27,unknown
119,425 mm Cavity Wall,K,0.27,425 mm cavity wall


In [16]:
ber["FirstWallType_Description"].unique()

array(['300mm Cavity', '300mm Filled Cavity', 'Stone',
       'Solid Mass Concrete', 'Concrete Hollow Block',
       '225mm Solid brick', 'Timber Frame', '325mm Solid Brick', 'Other',
       nan], dtype=object)

In [17]:
ber["wall_type"] = ber["FirstWallType_Description"].fillna("Unknown").replace({"Other": "Unknown"}).str.lower()

In [18]:
ber["wall_type"].unique()

array(['300mm cavity', '300mm filled cavity', 'stone',
       'solid mass concrete', 'concrete hollow block',
       '225mm solid brick', 'timber frame', '325mm solid brick',
       'unknown'], dtype=object)

In [19]:
ber

,Year_of_Construction,UValueWall,FirstWallType_Description,age_band_bin,age_band,wall_type
0,1995,0.33,300mm Cavity,"(1993.0, 1999.0]",H,300mm cavity
1,2002,0.55,300mm Filled Cavity,"(1999.0, 2004.0]",I,300mm filled cavity
2,1978,0.94,300mm Cavity,"(1977.0, 1982.0]",F,300mm cavity
3,1900,2.10,Stone,"(-inf, 1900.0]",A,stone
4,1983,0.54,Solid Mass Concrete,"(1982.0, 1993.0]",G,solid mass concrete
...,...,...,...,...,...,...
930331,1970,1.99,NaN,"(1966.0, 1977.0]",E,unknown
930332,1972,0.90,300mm Filled Cavity,"(1966.0, 1977.0]",E,300mm filled cavity
930333,1950,2.02,Stone,"(1949.0, 1966.0]",D,stone
930334,1998,0.69,NaN,"(1993.0, 1999.0]",H,unknown


In [20]:
ber_merged = ber[["age_band", "wall_type", "UValueWall"]].merge(wall_defaults[["age_band", "wall_type", "UValueWall_default"]])

In [21]:
ber_merged["UValueWall_is_default"] = ber_merged["UValueWall"] == ber_merged["UValueWall_default"]

In [22]:
ber_uvalue_wall_is_default = ber_merged.query("`UValueWall_is_default` == True")

In [23]:
ber_uvalue_wall_is_default

,age_band,wall_type,UValueWall,UValueWall_default,UValueWall_is_default
1,H,300mm cavity,0.55,0.55,True
2,H,300mm cavity,0.55,0.55,True
3,H,300mm cavity,0.55,0.55,True
4,H,300mm cavity,0.55,0.55,True
11,H,300mm cavity,0.55,0.55,True
...,...,...,...,...,...
928652,F,225mm solid brick,1.10,1.10,True
928653,F,225mm solid brick,1.10,1.10,True
928654,F,225mm solid brick,1.10,1.10,True
928655,F,225mm solid brick,1.10,1.10,True


In [24]:
number_of_uvalue_wall_default_hh = len(ber_uvalue_wall_is_default)
number_of_hh = len(ber)
percentage_of_uvalue_wall_default_hh = round(number_of_uvalue_wall_default_hh / number_of_hh * 100, 2)

In [25]:
display(f"{percentage_of_uvalue_wall_default_hh}% of Dublin BERPublicsearch hh use default values for walls")

'34.76% of Dublin BERPublicsearch hh use default values for walls'

# (Optional) Download BER default-linked data 

In [26]:
from IPython.display import FileLink

In [27]:
output_filepath = "../data/BERPublicsearch_with_default_uvaluewall.csv"

In [28]:
ber_merged.to_csv(output_filepath)

In [32]:
display(f"Click to download the default-linked data")
FileLink(output_filepath)

'Click to download the default-linked data'

/home/wsl-rowanm/Code/berpublicsearch-defaults/data/BERPublicsearch_with_default_uvaluewall.csv